1 - Снять видео на камеру (любую, можно телефона). Выполнить кручение видео с скоростью 10 градусов в секунду относительно центра (в любую сторону). 
2 - Видео после выполнения п.1 и вернуть его в исходное состояние. 
3 - Сконвертируйте видео в mp4 при помощи ffmpeg

In [1]:
import cv2
import time
import ffmpeg
import os
import subprocess
__name__ == "__main__"
FRAMES_PER_SECOND = 30
ROTATE_DEGREE_PER_SECOND = 10
#Откроем имеющийся файл
cap = cv2.VideoCapture("vid300.mp4")
current_rotate_angle = 0
addition_angle_per_frame = ROTATE_DEGREE_PER_SECOND/FRAMES_PER_SECOND


# Check if camera opened successfully
if (cap.isOpened()== False):
    print("Error opening video file")
#Прочитаем первый кадр чтобы узнать разрешение и вычислить центр
ret, frame = cap.read()
(h,w) = frame.shape[0:2]
center = (w//2,h//2)
fourcc = cv2.VideoWriter_fourcc(*'FMP4')
video=cv2.VideoWriter('doublevid.avi',fourcc,30,(w*2,h))
#Считываем кадр за кадром:
while(cap.isOpened()):
    #вычисляем 2 матрицы поворота, одна разворачивает, другая комплиментарная для возварата видео обратно
    matrix = cv2.getRotationMatrix2D(center,current_rotate_angle,1)
    matrix_compliment = cv2.getRotationMatrix2D(center,-current_rotate_angle,1)
    
#Выполняем чтение кадра:
    ret, frame = cap.read()
    if ret == True:
        #Если кадр считался, поворачиваем этот кадр:
        new_image = cv2.warpAffine(frame,matrix,(w,h)) 
        #Повернутый кадр разворачиваем обратно:
        anti_image = cv2.warpAffine(new_image,matrix_compliment,(w,h))
        #Соединяем 2 кадра в один:
        concat_image = cv2.hconcat([new_image, anti_image])
        cv2.imshow('frame',concat_image)
        #Записываем объединенный кадр в видео:
        video.write(concat_image)
        current_rotate_angle += addition_angle_per_frame        
        
    if cv2.waitKey(1) & 0xFF == ord('q') or ret==False :
        #Если кадр не считался, значит видео закончилось. Закрываем Видеокапче, закрываем окна кадров, завершаем видео
       cap.release()
       cv2.destroyAllWindows()
       video.release()
       print("everything is released")  
       break 
#ffmpeg.input("d:/doublevid.avi").output("d:/doublevid2.mp4")            
#command = ["ffmpeg", "-i", "d:/doublevid.avi", "-c:v", "h264", "-c:a", "copy", "d:/doublevid2.mp4"]
#subprocess.run(command, check=True)    
    
def convert_avi_to_mp4(input_file, output_file):    
    
    if not os.path.exists(input_file):
        print(f"Входной файл {input_file} не найден.")
        return
    
    command = [
        "ffmpeg",
        "-i", input_file,
        "-c:v", "libx264",
        "-crf", "18",
        "-c:a", "aac",
        "-b:a", "128k",
        output_file
    ]

    # Вызываем ffmpeg и ожидаем завершения процесса
    subprocess.run(command, check=True)
    print(f" Выходной файл: {output_file}")

# Вызываем процедуру:
input_file = "doublevid.avi"
output_file = "output.mp4"
convert_avi_to_mp4(input_file, output_file)

everything is released
Перевод завершен. Выходной файл: output.mp4


Выводы:  При помощи VideoCapture можно видео разбить на кадры.  В данном видео мы создали матрицу афинных преобразований
из матрицы поворота. Повернули видео со скоростью 10 градусов в секунду и обратно. Полученные кадры "упаковали " в видео.

<video controls src = "https://drive.google.com/file/d/1r4Y6iHwdxpY1BEP8HzwFuJ2BaXG4Mc-L/view?usp=drive_link" width = "854" height = "300">